In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import uproot_methods
import networkx as nx

In [ ]:
g = pickle.load(open("graph_1.pkl", "rb"))

In [ ]:
def has_reco(g, particle):
    for edge in g.edges(particle):
        if edge[1][0] in ["tower", "track"]:
            return True
    return False

In [ ]:
particles = [n for n in g.nodes if n[0] == "particle"]
particles_with_reco = [n for n in particles if has_reco(g, n)]

In [ ]:
etas = np.array([g.nodes[n]["eta"] for n in particles_with_reco])
phis = np.array([g.nodes[n]["phi"] for n in particles_with_reco])
energys = np.array([g.nodes[n]["energy"] for n in particles_with_reco])

etas_tower = np.array([g.nodes[n]["eta"] for n in g.nodes if n[0] == "tower"])
phis_tower = np.array([g.nodes[n]["phi"] for n in g.nodes if n[0] == "tower"])
energys_tower = np.array([g.nodes[n]["energy"] for n in g.nodes if n[0] == "tower"])

etas_track = np.array([g.nodes[n]["eta"] for n in g.nodes if n[0] == "track"])
phis_track = np.array([g.nodes[n]["phi"] for n in g.nodes if n[0] == "track"])
energys_track = np.array([g.nodes[n]["pt"] for n in g.nodes if n[0] == "track"])

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(etas, phis, alpha=0.5, marker=".", label="GenParticle")
plt.scatter(etas_tower, phis_tower, alpha=0.5, marker="s", label="Tower")
plt.scatter(etas_track, phis_track, marker="x", label="Track")

plt.ylim(-4,4)
plt.legend(frameon=False)
plt.xlabel("eta")
plt.ylabel("phi")

In [ ]:
matrices = pickle.load(open("out.pkl", "rb"))

In [ ]:
plt.figure(figsize=(10,10))

X = matrices["X"][0]
ygen = matrices["ygen"][0]
ycand = matrices["ycand"][0]
plt.scatter(ygen[:, 3], ygen[:, 4], marker=".")
plt.scatter(ycand[:, 3], ycand[:, 4], marker="x")
plt.scatter(X[:, 2], X[:, 3], marker="s", alpha=0.5)


In [ ]:
ygen = np.concatenate(matrices["ygen"], axis=0)
ycand = np.concatenate(matrices["ycand"], axis=0)

In [ ]:
msk = (ygen[:, 0] == 1) & (ycand[:, 0] == 1)
plt.figure(figsize=(5,5))
plt.hist((ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-5,5,100));
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("Charged hadrons")

In [ ]:
msk = (ygen[:, 0] == 2) & (ycand[:, 0] == 2)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 5]-ycand[msk, 5])/ygen[msk, 5], bins=np.linspace(-5,5,100));
plt.xlabel("E resolution (gen-pf)/gen")
plt.title("neutral hadrons")

In [ ]:
msk = (ygen[:, 0] == 3) & (ycand[:, 0] == 3)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 5]-ycand[msk, 5])/ygen[msk, 5], bins=np.linspace(-5,5,100));
plt.xlabel("E resolution (gen-pf)/gen")
plt.title("photons")

In [ ]:
msk = (ygen[:, 0] == 4) & (ycand[:, 0] == 4)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-0.5, 0.5, 100))
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("muons")

In [ ]:
msk = (ygen[:, 0] == 5) & (ycand[:, 0] == 5)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-0.5, 0.5, 100))
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("electrons")

In [ ]:
import sklearn
import sklearn.metrics

In [ ]:
confusion = sklearn.metrics.confusion_matrix(
    ygen[:, 0], ycand[:, 0], normalize="true"
)

In [ ]:
msk_a = ycand[:, 0] == 0
msk_b = ygen[:, 0] == 2

np.sum(msk_a & msk_b)

In [ ]:
plt.imshow(confusion, cmap="Blues")
plt.title("Reconstructed PID (normed to gen)")
plt.xlabel("Delphes PF PID")
plt.ylabel("Gen PID")
plt.xticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.yticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.colorbar()